In [97]:
import os
import rawpy
import imageio
from PIL import Image
import shutil
import osf

/home/niklas/anaconda3/envs/oads/lib/python3.9/site-packages/modgrammar/__init__.py:1564: FutureWarning: Possible nested set at position 1
  cls.regexp = re.compile(regexp)
/home/niklas/anaconda3/envs/oads/lib/python3.9/site-packages/modgrammar/__init__.py:1564: FutureWarning: Possible nested set at position 9
  cls.regexp = re.compile(regexp)


In [87]:
import numpy as np

In [39]:
oads_dir = '/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup'
arw_dir = os.path.join(oads_dir, 'ARW')
jpeg_dir = os.path.join(oads_dir, 'JPG')
supervisely_dir = os.path.join(oads_dir, 'Supervisely')

In [86]:
len(os.listdir(arw_dir))

6130

In [84]:
len(os.listdir(jpeg_dir))

6130

In [85]:
len(os.listdir(supervisely_dir))

6130

In [92]:
jpeg_filenames = os.listdir(jpeg_dir)
supervisely_filenames = os.listdir(supervisely_dir)
arw_filenames = os.listdir(arw_dir)

In [83]:
# for x in set([x for x in jpeg_filenames]):
#     arw_filename = f"{x.split('.')[0]}.ARW"
#     if arw_filename not in arw_filenames:
#         os.unlink(os.path.join(jpeg_dir, x))

# for x in set([x for x in supervisely_filenames]):
#     arw_filename = f"{x.split('.')[0]}.ARW"
#     if arw_filename not in arw_filenames:
#         os.unlink(os.path.join(supervisely_dir, x))
    
# for x in [x for x in os.listdir(supervisely_dir) if x.split('.')[1] != "JPG"]:
#     if os.path.join(supervisely_dir, f"{x.split('.')[0]}.JPG") in os.listdir(supervisely_dir):
#         os.unlink(os.path.join(supervisely_dir, x))
#     else:
#         os.rename(os.path.join(supervisely_dir, x), os.path.join(supervisely_dir, f"{x.split('.')[0]}.JPG"))

In [21]:
jpeg_ids = [x.split('.')[0] for x in os.listdir(jpeg_dir)]
supervisely_ids = [x.split('.')[0] for x in os.listdir(supervisely_dir)]

In [23]:
missing_jpeg = []
missing_supervisely = []
for x in os.listdir(arw_dir):
    image_name = x.split('.')[0]
    if image_name not in jpeg_ids:
        missing_jpeg.append(image_name)
    if image_name not in supervisely_ids:
        missing_supervisely.append(image_name)

In [31]:
import tqdm

In [36]:
is_indoor = False

In [46]:
counter = 0
for x in tqdm.tqdm(missing_supervisely):
    jpeg_path = os.path.join(jpeg_dir, f"{x}.JPG")
    arw_path = os.path.join(arw_dir, f"{x}.ARW")
    supervisely_path = os.path.join(supervisely_dir, f"{x}.JPG")
    # print(jpeg_path)
    with Image.open(jpeg_path) as im:
        w, h = im.size # get width, heigth
        # if (w, h) != (5496, 3672):
        #     # Recompute JPEG
        #     with rawpy.imread(arw_path) as raw:
        #         if is_indoor:
        #             rgb = raw.postprocess(no_auto_bright=True, gamma=(10,10))
        #         else:
        #             rgb = raw.postprocess()
        #         imageio.imsave(jpeg_path, rgb) # save in JPG folder

            # print(w,h)
            # counter += 1
        new_w = int(w/4) # resize by factor 4
        new_h = int(h/4)
        # if (new_h, new_w) != (918, 1374):
        #     print(new_h, new_w)
        imResize = im.resize((new_w,new_h), Image.Resampling.LANCZOS) 
        imName = supervisely_path
        imResize.save(imName, 'JPEG') # save in the seperate Supervisely folder

100%|██████████| 1478/1478 [07:22<00:00,  3.34it/s]


In [33]:
counter

1117

## Dataverse 

In [51]:
!pip install pyDataverse

In [59]:
from pyDataverse.models import Dataset
from pyDataverse.utils import read_file
from pyDataverse.api import NativeApi
api = NativeApi('https://dataverse.harvard.edu', '778628b9-2a95-4e65-891e-1cbc8be0fe78')

In [62]:
api.get_info_version().json()

{'status': 'OK', 'data': {'version': '5.13', 'build': '1244-79d6e57'}}

In [63]:
from pyDataverse.models import Datafile, Dataset
df = Datafile()

In [64]:
ds_pid = 'doi:10.7910/DVN/YNSDOU'

In [72]:
import os

In [73]:
oads_dir = '/home/niklas/projects/data/oads/oads_arw/ARW/'
oads_images = [os.path.join(oads_dir, x) for x in os.listdir(oads_dir)]

In [87]:
for image_path in oads_images:
    df.set({"pid": ds_pid, "filename": image_path})
    # df.get()
    resp = api.upload_datafile(ds_pid, image_path, df.json())
    resp.json()

KeyboardInterrupt: 

# OSF

In [1]:
osf_dir = '/home/niklas/projects/osf_oads/'

In [2]:
import osfclient as osf

In [3]:
_osf = osf.OSF(username='n.muller@uva.nl', token='zADpOUgHPW1UWttO2RQR0CciFLZCZyCFPmlJti7bPgfkm8RcLX1yBeuNgaIQKzwtC7GaYK')

In [4]:
project = _osf.project('2xzdf')

In [50]:
project.storage()

KeyboardInterrupt: 

In [5]:
from osfclient import cli

In [6]:
class args:

    def __init__(self, project, username=None, update=False, token=None, force=False, destination=None, source=None, recursive=False, target=None, output=None, remote=None, local=None):
        self.project = project
        self.username = username
        self.token = token
        self.update = update # applies to upload, clone, and fetch
        self.force = force # applies to fetch and upload 
        # upload arguments:
        self.destination = destination
        self.source = source
        self.recursive = recursive
        # remove argument:
        self.target = target
        # clone argument:
        self.output = output
        # fetch arguments:
        self.remote = remote
        self.local = local 

In [7]:
arguments = args(
    username='n.muller@uva.nl',
    project='2xzdf',
    token='zADpOUgHPW1UWttO2RQR0CciFLZCZyCFPmlJti7bPgfkm8RcLX1yBeuNgaIQKzwtC7GaYK',
    # upload arguments:
    destination='', 
    source='',
    # remove argument:
    target='',
    # clone argument:
    output='',
    # fetch arguments:
    remote='',
    local=''
)

In [8]:
cli.list_(arguments)

RuntimeError: Response has status code 502 not (200,)

In [ ]:
cli.

# OLD

In [26]:
# os.listdir(/C:\Users\nikla\OADS\OADS Missing Images from Camera\Upload_to_drive\ARW)
counter = 0
for x in os.listdir('/mnt/c/Users/nikla/OADS/OADS Missing Images from Camera/Upload_to_drive_DONE/ARW'):
    if x.endswith('.ARW'):
        if x not in os.listdir('/home/niklas/projects/data/oads/oads_arw/ARW'):
            print(x)
            # shutil.copy(f'/mnt/c/Users/nikla/OADS/OADS Missing Images from Camera/Upload_to_drive/ARW/{x}', '/home/niklas/projects/data/oads/oads_arw/ARW')

In [23]:
counter

15

In [2]:
all_arw = os.listdir('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/ARW')
all_jpeg_images = os.listdir('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/JPG')
# all_arw = os.listdir('/home/niklas/projects/data/oads/oads_annotations/')

In [5]:
folders = os.listdir('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/oads_annotations')
all_jpeg = []
all_jpeg_names = {}
for folder in folders:
    if os.path.isdir(os.path.join('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/oads_annotations', folder)):
        all_jpeg_names[folder] = os.listdir(os.path.join('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/oads_annotations', folder, 'ann'))
        all_jpeg.extend(os.listdir(os.path.join('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/oads_annotations', folder, 'ann')))


In [3]:
all_arw = [x.split('.')[0] for x in all_arw]

In [6]:
all_jpeg = [x.split('.')[0] for x in all_jpeg]
all_jpeg_images = [x.split('.')[0] for x in all_jpeg_images]

In [ ]:
# all_jpeg_names = [x.split('.')[0] for x in all_jpeg_names]

In [7]:
len(all_arw), len(all_jpeg), len(all_jpeg_images)

(6130, 5691, 5646)

In [15]:
both_jpg_and_raw = []
raw_but_not_jpeg = []
jpeg_but_not_raw = []

jpeg_but_not_raw_names = {name: [] for name in all_jpeg_names.keys()}

for x in all_arw:
    if x in all_jpeg:
        both_jpg_and_raw.append(x)
    else:
        raw_but_not_jpeg.append(x)

for x in all_jpeg:
    if x not in all_arw:
        jpeg_but_not_raw.append(x)

for name, xs in all_jpeg_names.items():
    for x in xs:
        id = x.split('.')[0]
        if id not in all_arw:
            jpeg_but_not_raw_names[name].append(x)

In [16]:
len(both_jpg_and_raw), len(raw_but_not_jpeg), len(jpeg_but_not_raw)

(5477, 653, 214)

In [10]:
pathARW = '/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/ARW'

In [18]:
# Need annotation
new_dir = '/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/Need_annotations'
os.makedirs(new_dir, exist_ok=True)

supervisely_filenames = os.listdir('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/Supervisely')
jpeg_filenames = os.listdir('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/JPG')

for image_id in all_arw:
    # Is in JPG?
    if f'{image_id}.jpg' not in jpeg_filenames:
        new_jpg_filename = f'{image_id}.jpg'
        if f'{image_id}.JPG' not in jpeg_filenames:
            new_jpg_filename = f'{image_id}.JPG'
    else:
        new_jpg_filename = None

    
    # Is in Supervisely?
    if f'{image_id}.jpg' in supervisely_filenames:
        new_supervisely_filename = f'{image_id}.jpg'
        if f'{image_id}.JPG' in supervisely_filenames:
            new_supervisely_filename = f'{image_id}.JPG'
    else:
        new_supervisely_filename = None

    # Has Annotation?
    if image_id in raw_but_not_jpeg:
        # Needs annotation - Supervisely image
        needs_annotation_filename = f'{image_id}.JPG'
    else:
        needs_annotation_filename = None

    if new_supervisely_filename is not None or new_jpg_filename is not None or needs_annotation_filename is not None:
        # Get raw image
        with rawpy.imread(pathARW+'/'+f'{image_id}.ARW') as raw:
            rgb = raw.postprocess()

        # Save to JPG folder (jpeg compressed but full resolution)
        if new_jpg_filename is not None:
            imageio.imsave(os.path.join('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/JPG', new_jpg_filename), rgb) # save in JPG folder

        # Downsample
        im = Image.fromarray(rgb)
        w, h = im.size # get width, heigth
        new_w = int(w/4) # resize by factor 4
        new_h = int(h/4)
        imResize = im.resize((new_w,new_h), Image.Resampling.LANCZOS) 

        # Save to Supervisely folder
        if new_supervisely_filename is not None:
            imName = os.path.join('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/Supervisely', new_supervisely_filename)
            imResize.save(imName, 'JPEG') # save in the seperate Supervisely folder

        # Save to needs annotation folder
        if needs_annotation_filename is not None:
            imName = os.path.join(new_dir, needs_annotation_filename)
            imResize.save(imName, 'JPEG') # save in the seperate Supervisely folder



    # if f'{image_id}.JPG' in supervisely_filenames:
    #     # Copy from here
    #     shutil.copy(src=os.path.join('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/Supervisely', f'{image_id}.JPG'), dst=os.path.join(new_dir, f'{image_id}.JPG'))
    #     # print(image_id)
    # elif f'{image_id}.jpg' in supervisely_filenames:
    #     # Copy from here
    #     shutil.copy(src=os.path.join('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/Supervisely', f'{image_id}.jpg'), dst=os.path.join(new_dir, f'{image_id}.jpg'))
    #     # print(image_id)
    # else:
    #     # Make JPEG and copy then
    #     with rawpy.imread(pathARW+'/'+f'{image_id}.ARW') as raw:
    #         rgb = raw.postprocess()
    #     imagename = f'{image_id}.JPG'
    #     if imagename not in jpeg_filenames and f'{image_id}.jpg' not in jpeg_filenames:
    #         imageio.imsave(os.path.join('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/JPG', imagename), rgb) # save in JPG folder

    #     im = Image.fromarray(rgb)
    #     w, h = im.size # get width, heigth
    #     new_w = int(w/4) # resize by factor 4
    #     new_h = int(h/4)
    #     imResize = im.resize((new_w,new_h), Image.Resampling.LANCZOS) 
    #     imName = os.path.join(new_dir, f'{image_id}.JPG')
    #     imResize.save(imName, 'JPEG') # save in the seperate Supervisely folder



In [13]:
folders = os.listdir('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/oads_annotations')
all_jpeg = []
for folder in folders:
    if os.path.isdir(os.path.join('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/oads_annotations', folder)):
        all_jpeg.extend(os.listdir(os.path.join('/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/oads_annotations', folder, 'ann')))


all_jpeg = [x.split('.')[0] for x in all_jpeg]

new_dir = '/mnt/c/Users/nikla/OneDrive - UvA/OADS_Backup/Need_annotations'
os.makedirs(os.path.join(new_dir, 'duplicates'), exist_ok=True)

for x in os.listdir(new_dir):
    if not os.path.isdir(x) and x.split('.')[0] in all_jpeg:
        # both_jpg_and_raw.append(x)
        print(f'Moving {x}')
        shutil.move(src=os.path.join(new_dir, x), dst=os.path.join(new_dir, 'duplicates', x))
    # else:
    #     raw_but_not_jpeg.append(x)

# for x in all_jpeg:
#     if x not in all_arw:
#         jpeg_but_not_raw.append(x)

# for name, xs in all_jpeg_names.items():
#     for x in xs:
#         id = x.split('.')[0]
#         if id not in all_arw:
#             jpeg_but_not_raw_names[name].append(x)

Moving 000a464c4c86d4cc.JPG
Moving 0100038315934a8e.JPG
Moving 01e4692c8c8283c0.JPG
Moving 022b0912989013ea.JPG
Moving 034344564280fef8.JPG
Moving 04004044c6840cf8.JPG
Moving 0406361461b5c4e0.JPG
Moving 0505454d4970e3b2.JPG
Moving 06c0cc6c8555f496.JPG
Moving 07088c9d0c0682f0.JPG
Moving 078766364cf676e0.JPG
Moving 090b656571511100.JPG
Moving 098d1d0c0e6fabfc.JPG
Moving 09e4479736ecdef8.JPG
Moving 0a2c0c3a1a04a2fa.JPG
Moving 0b2a8699c8ec9cd8.JPG
Moving 0c3c109132288484.JPG
Moving 0e1a5bd3308e6038.JPG
Moving 0f9f9787c92dc5c0.JPG
Moving 11060e888ccc04c6.JPG
Moving 11313b624c94f4c8.JPG
Moving 125a425644b498a0.JPG
Moving 1311b1b1e52432a0.JPG
Moving 159195d4e169f0a0.JPG
Moving 160e0e4de9f17b8f.JPG
Moving 1682892694153b9e.JPG
Moving 189801c4d1f1c3ff.JPG
Moving 19181919959e0f86.JPG
Moving 1a1e1c98d2580091.JPG
Moving 1a22287bfb593940.JPG
Moving 1a32c0c868b49098.JPG
Moving 1a3a360dd591ecec.JPG
Moving 1b0ccdcdcdb42cc0.JPG
Moving 1b95d508ececede0.JPG
Moving 1dacf432f320068c.JPG
Moving 1e1e12d2f2323

In [43]:
[name for name, items in jpeg_but_not_raw_names.items() if len(items) > 0]
# len(jpeg_but_not_raw_names['Corner_cases'])

['Corner_cases', 'L_002_2', 'L_001', 'R_000', 'h_003', 'Unknown_001']

In [58]:
len(jpeg_but_not_raw_names['Unknown_001'])

1